In [15]:
import os
import re
import pandas as pd
import numpy as np

In [16]:
dir_path = r"E:\BRISTOL COURSE\TB2\mini project bristol\JPMorgan_Set01\Tapes"
file_ls = os.listdir(dir_path)  #get every name of the file
file_ls[:5]

['UoB_Set01_2025-01-02tapes.csv',
 'UoB_Set01_2025-01-03tapes.csv',
 'UoB_Set01_2025-01-06tapes.csv',
 'UoB_Set01_2025-01-07tapes.csv',
 'UoB_Set01_2025-01-08tapes.csv']

In [17]:
len(file_ls)

125

In [68]:
#found the max and min deal price
def detailprice(name):
    date=[]
    segments = []
    tapes = pd.read_csv(r"E:/BRISTOL COURSE/TB2/mini project bristol/JPMorgan_Set01/Tapes/" + name + "", header=None,
                        names=["Times", "price", "volume"])
    tapes['price'] = pd.to_numeric(tapes['price'], errors='coerce')
    segment_size = len(tapes) // 50
    data_pattern = r'\d{4}-\d{2}-\d{2}'
    match = re.search(data_pattern, name)
    date.append(match.group())
    for i in range(50):
        start_idx = i * segment_size
        # 对于最后一个段，取所有剩余的数据点
        end_idx = start_idx + segment_size if i < 49 else len(tapes)

        segment = tapes.iloc[start_idx:end_idx]
        max_price = segment['price'].max()
        min_price = segment['price'].min()
        open_price = segment['price'].iloc[0]
        close_price = segment['price'].iloc[-1]
        segments.append({
            'period': date[0]+f"-{i+1}",
            'max_price': max_price,
            'min_price': min_price,
            'open_price': open_price,
            'close_price': close_price
        })
        segments_df = pd.DataFrame(segments)
    return segments_df

In [69]:
# 初始空DataFrame，用于定义列名
result_df = pd.DataFrame(columns=['period', 'max_price', 'min_price', 'open_price', 'close_price'])
# 用于存储每个文件处理结果的DataFrame列表
df_list = []
for i in file_ls:
    segments_df = detailprice(i)  # 调用detailprice函数处理每个文件
    df_list.append(segments_df)  # 将结果添加到列表中
# 在循环结束后，一次性拼接所有DataFrame
result_df = pd.concat(df_list, axis=0, ignore_index=True)  # 使用ignore_index=True重置索引
# 现在 result_df 包含了所有文件处理后的数据，无需再创建新的DataFrame
result_df

,period,max_price,min_price,open_price,close_price
0,2025-01-02-1,270,249,267,261
1,2025-01-02-2,269,248,264,266
2,2025-01-02-3,270,248,269,254
3,2025-01-02-4,269,247,255,265
4,2025-01-02-5,267,247,262,261
...,...,...,...,...,...
6245,2025-07-01-46,113,93,109,110
6246,2025-07-01-47,113,93,109,109
6247,2025-07-01-48,113,93,109,107
6248,2025-07-01-49,113,93,107,98


In [70]:
result_df.to_csv("sperated_tapes.csv")

In [54]:
data_pattern = r'\d{4}-\d{2}-\d{2}'  # found the date in the file
date = []
for i in file_ls:
    match = re.search(data_pattern, i)
    date.append(match.group())
date = pd.to_datetime(date)
date

DatetimeIndex(['2025-01-02', '2025-01-03', '2025-01-06', '2025-01-07',
               '2025-01-08', '2025-01-09', '2025-01-10', '2025-01-13',
               '2025-01-14', '2025-01-15',
               ...
               '2025-06-18', '2025-06-19', '2025-06-20', '2025-06-23',
               '2025-06-24', '2025-06-25', '2025-06-26', '2025-06-27',
               '2025-06-30', '2025-07-01'],
              dtype='datetime64[ns]', length=125, freq=None)

In [14]:
file_ls_series = pd.Series(file_ls)
results_df = []


def apply_detailprice(filename):
    return detailprice(filename)  # Make sure that this function returns a tuple or list of four values


results_df = file_ls_series.apply(apply_detailprice).apply(pd.Series)
results_df.columns = ['max_price', 'min_price', 'open_price', 'close_price']
date = pd.Series(date)
results_df.insert(0, "date", date)
results_df.head(5)



,date,max_price,min_price,open_price,close_price
0,2025-01-02,303,243,267,288
1,2025-01-03,317,270,281,291
2,2025-01-06,346,276,278,327
3,2025-01-07,388,317,336,369
4,2025-01-08,374,303,365,311


In [15]:
start_date, end_date = date.min(), date.max()  # create a new dataframe which have all date in this range
date_range = pd.date_range(start=start_date, end=end_date)
date_range = pd.Series(date_range)
df_full = pd.DataFrame(date_range, columns=['date'])
df_full.head(5)

,date
0,2025-01-02
1,2025-01-03
2,2025-01-04
3,2025-01-05
4,2025-01-06


In [16]:
df_merged = pd.merge(df_full, results_df, on="date", how="left")  # left join 2 df
df_merged.head(5)

,date,max_price,min_price,open_price,close_price
0,2025-01-02,303.0,243.0,267.0,288.0
1,2025-01-03,317.0,270.0,281.0,291.0
2,2025-01-04,NaN,NaN,NaN,NaN
3,2025-01-05,NaN,NaN,NaN,NaN
4,2025-01-06,346.0,276.0,278.0,327.0


In [19]:
df_merged.to_csv("E:/mini project bristol/processed data(LOBS,tapes)/processed_tapes.csv")